## Define Dynamic Parameters

In [11]:
file_no = 0
my_seed = 64

## Import Packages

In [12]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import glob
import os
from sklearn.model_selection import  train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import ntpath
import csv

tf.random.set_seed(my_seed)
np.random.seed(my_seed)

2.1.0
2.2.4-tf


NameError: name 'dsf' is not defined

## Import CSVs

In [ ]:
ROOT_DIR = os.path.abspath(os.curdir)
path = ROOT_DIR + '\\..\\..\\Datasets\\processed\\regression_10d'
print(path)

csvfiles = glob.glob(path + "/*.csv")
dfs = []

for file in csvfiles:
    df = pd.read_csv(file)
    df.name = (ntpath.basename(file)).split(".",1)[0]
    dfs.append(df)

df = dfs[file_no]
print("Stock: ")
print(df.name)
print(my_seed)

# Splitting dataset in dependent and independent variables

In [ ]:
x = np.array(df.iloc[:, 1:-1].values, dtype=np.float)
#x = np.array(df.iloc[:, 0:-1].values)
y = np.array(df.iloc[:, -1:].values)


# Splitting dataset (training + testing)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, shuffle=False, random_state=32)

print("Train - Independent Variables")
print(x_train)
print("Test - Independent Variables")
print(x_test)

# Building ANN with MLP

In [ ]:
ann = tf.keras.models.Sequential()
# hidden layers
ann.add(tf.keras.layers.Dense(units=9, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
# output layer
ann.add(tf.keras.layers.Dense(units=1, activation='linear'))

optimizer = tf.keras.optimizers.Adam()
ann.compile(optimizer = optimizer, loss = 'mean_squared_error')

ann.fit(x_train, y_train, batch_size = 32, epochs = 200)

weights = ann.get_weights()

# Predicting Test Set Results

In [ ]:
y_pred = ann.predict(x_test)
# boolean result of whether predicted probability is larger than 0.5
# result > 0.5 = 1, result <= 0.5 = 0
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))


# Obtaining Confusion Matrix and Accuracy Score for predictions

In [ ]:
from sklearn.metrics import mean_absolute_error


print()
print("Mean Absolute Error: ")
mean_absolute_error(y_test, y_pred)

# Classifying results

In [ ]:
prediction_arr = []
actuals_arr = []
neutral_percentage = 0.002

for i in range(1, len(y_pred)):
    if y_pred[i] - y_test[i-1] > (y_pred[i] * neutral_percentage):
            prediction_arr.append([1,0,0])
    elif y_pred[i] - y_test[i-1]  < - (y_pred[i] * neutral_percentage):
            prediction_arr.append([0,0,1])
    else:
            prediction_arr.append([0,1,0])

for i in range(1, len(y_test)):
    if y_test[i] - y_test[i-1] > (y_test[i] * neutral_percentage):
            actuals_arr.append([1,0,0])
    elif y_test[i] - y_test[i-1]  < - (y_test[i] * neutral_percentage):
            actuals_arr.append([0,0,1])
    else:
            actuals_arr.append([0,1,0])

actuals_arr = np.array(actuals_arr)
prediction_arr = np.array(prediction_arr)



# Results

In [ ]:
%%capture cap

from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

# Confusion Matrix: Columns = 0, 1 (Predicted Label)
# Confusion Matrix: Rows = 0, 1 (Actual Label)
cm = multilabel_confusion_matrix(np.argmax(actuals_arr, axis=1), np.argmax(prediction_arr, axis=1))
print(cm)
print()

print("Accuracy Score: ")
accuracy_score = accuracy_score(actuals_arr, prediction_arr)
print(accuracy_score)

# Graphs

In [ ]:
%matplotlib inline

output_file_name = df.name + '_' + str(my_seed)
data = [output_file_name, accuracy_score]

with open(ROOT_DIR+'\\regression_results\\results.csv', 'a', newline="") as f:
    writer = csv.writer(f)
    writer.writerow(data)

with open(ROOT_DIR+'\\regression_results\\'+ output_file_name +'.txt', 'w') as f:
    f.write(cap.stdout)

    
test_nos = []
for i, val in enumerate(y_test):
    test_nos.append(i)
plt.plot(test_nos, y_test)
plt.plot(test_nos, y_pred, color='green')